In [ ]:
!gshell init

In [ ]:
!gshell info --with-id 17axH-dYh-Wss_ec92ugSl8AmSuZS-xn9

In [ ]:
import signal
from pathlib import Path
import wget

parser_version = '1.6.4'
url = f'https://github.com/nemoware/document-parser/releases/download/{parser_version}/document-parser-{parser_version}.jar'
if not Path(f'document-parser-{parser_version}.jar').is_file():
    wget.download(url)

In [ ]:
import glob
import zipfile

with zipfile.ZipFile(f'./{glob.glob("ДД по практикам*.zip")[0]}',
                     'r') as zip_ref:
    zip_ref.extractall('./')

In [24]:
parser_version = '1.6.4'
!java -cp "document-parser-$parser_version/classes;document-parser-$parser_version/lib/*" com.nemo.document.parser.App -i "ДД по практикам\Практика правового сопровождения земельных отношений и сделок с недвижимым имуществом\ДС 2.docx"

{"documents":[{"documentType":"AGREEMENT","paragraphs":[{"paragraphHeader":{"offset":0,"text":"Соглашение № 1\r\nО расторжении договора аренды № Р15 123597000045Р  \r\nот \"01\" июня 2019 года","length":91},"paragraphBody":{"offset":87,"text":"г. Томск                                                                                                \"31\"августа 2021 года\r\nООО «Ромашка» в лице генерального директора Иванова Иван Ивановича, действующего на основании Устава, именуемое в дальнейшем «Арендодатель», с одной стороны, ООО «Газпромнефть -  Центр»  в лице Соловьева Рината Геннадьевича,  действующего на основании доверенности  №Д-27 от 23.12.2019 г., именуемое в дальнейшем «Арендатор»,   вместе именуемые \"Стороны\", а индивидуально – \"Сторона\", подписали настоящее соглашение (далее - Соглашение) к договору аренды нежилого помещения  № Р15 123597000045Р  от \"01\" июня 2019  года (далее – Договор) о нижеследующем:\r\n1. Расторгнуть Договор с даты подписания данного соглашения.\

In [ ]:
import subprocess

s = [
    "gshell",
    "init"
]

subprocess.run(s, stdin=subprocess.PIPE, stdout=subprocess.PIPE)

In [27]:
import platform
import sys
import os
import json
import subprocess
import pandas as pd
import fnmatch
import base64
import requests
import time
import re

index = 1
number_of_docs = 0
parser_version = '1.6.4'
root = "ДД по практикам"

arrOfAllDocs = []
result = []
result_of_fail = []
result_of_possible = []
paragraph_number = 1
s = [
    "java",
    "-jar",
    f"document-parser-{parser_version}.jar",
    "--server.port=8083"
]
headers = {
    'Content-type': 'application/json',
    'Accept': 'application/json; text/plain'
}

for root, dirnames, filenames in os.walk('ДД по практикам'):
    if len(root.split('\\')) == 1: continue
    flag = False
    for i in root.split('\\'):
        if str(i).startswith('Исключена'):
            flag = True
            break
    if flag: continue
    for filename in fnmatch.filter(filenames, '*.docx'):
        arrOfAllDocs.append(os.path.join(root, filename))
    for filename in fnmatch.filter(filenames, '*.doc'):
        arrOfAllDocs.append(os.path.join(root, filename))

print(
    "Запуск document-parser на 8083 порту, если что0то пойдет не так, то руками УБЕЙТЕ java процесс"
)
java_subprocess = subprocess.Popen(s, creationflags=subprocess.CREATE_NEW_PROCESS_GROUP,
                                   stdout=subprocess.PIPE, encoding="utf-8")


def find_let(document):
    key_value = ['о нижеследующем:', 'нижеследующем:', 'о нижеследующем', 'нижеследующем']
    result = ""
    for i, p in enumerate(document['paragraphs']):
        if any(f.lower() in p['paragraphBody']['text'].lower() or f.lower() in
               p['paragraphHeader']['text'].lower() for f in
               key_value):
            text = ""

            for x in key_value:
                if x in p['paragraphBody']['text'].lower():
                    text += p['paragraphBody']['text'].split(x)[1]
                    break

            if text == "": text += p['paragraphBody']['text']

            text += "".join(
                str(x['paragraphBody']['text']) if re.match("^ *\d?[.] ", str(
                    x['paragraphBody']['text'])) or re.match("^ *\d?[.] ", str(
                    x['paragraphHeader']['text'])) else '' for x in
                document['paragraphs'][i:i + 4])

            textHeader = p['paragraphHeader']['text'] + "\n".join(
                str(x['paragraphHeader']['text']) if re.match("^ *\d?[.] ", str(
                    x['paragraphBody']['text'])) or re.match("^ *\d?[.] ", str(
                    x['paragraphHeader']['text'])) else '' for x in document['paragraphs'])
            result = {
                "path": docs,
                "name": docs.split("\\")[-1],
                "documentType": document['documentType'],
                "offset": p['paragraphBody']['offset'],
                "text": text,
                "length": len(text),
                "offsetHeader": p['paragraphHeader']['offset'],
                "textHeader": textHeader,
                "lengthHeader": len(textHeader)
            }
            break
    return result


time.sleep(2)
i = 1
while True:
    time.sleep(0.1)
    output_log_spring = java_subprocess.stdout.readline()
    sys.stdout.write("\rПроверка соединения #%i" % i)
    sys.stdout.flush()
    i += 1
    if output_log_spring.find(
            "Started DocumentParserService") != -1:
        print("\nГотово")
        java_subprocess.stdout.close()
        break

print("Запустился успешно")
print("Общее количество документов =", len(arrOfAllDocs))

for docs in arrOfAllDocs:
    try:
        file = open(docs, 'rb')
        encoded_string = base64.b64encode(file.read())
        encoded_string = str(encoded_string)[2:-1]
    except Exception as e:
        print(f"\nОшибка в файле {docs}")
        print(f"при конвертации в base64, исключение = {e.msg}")
        print("=" * 200)
        continue

    response = requests.post(
        "http://localhost:8083/document-parser",
        data=json.dumps({
            "base64Content": encoded_string,
            "documentFileType": docs.split(".")[-1].upper()
        }),
        headers=headers
    )
    resArr = []
    try:
        resArr = response.json()['documents']
    except Exception as e:
        print(f"\nОшибка в файле {docs}")
        print(f"Ответ от парсера {response.json()}")
        print(f"Исключение = {e}")
        print("=" * 200)
        continue

    sys.stdout.write(f"\rПроверка документа под номером {index}")
    sys.stdout.flush()
    index += 1

    document = []
    if resArr:
        document = resArr[0]
    else:
        continue

    if document['documentType'] == "CONTRACT":
        flag = False
        for i, p in enumerate(document['paragraphs']):
            if any(z.lower() in p['paragraphHeader']['text'].lower() for z in [
                'ПРЕДМЕТ', 'Общие состояние']) and p['paragraphBody']['length'] > 20:
                result.append(
                    {
                        "path": docs,
                        "name": docs.split("\\")[-1],
                        "documentType": document['documentType'],
                        "offset": p['paragraphBody']['offset'],
                        "text": p['paragraphBody']['text'],
                        "length": p['paragraphBody']['length'],
                        "offsetHeader": p['paragraphHeader']['offset'],
                        "textHeader": p['paragraphHeader']['text'],
                        "lengthHeader": p['paragraphHeader']['length']
                    })
                flag = True
                break
        if flag: continue
        obj = find_let(document)
        if obj != "":
            result.append(obj)
            flag = True

        if flag: continue
        result_of_fail.append(
            {
                "path": docs,
                "name": docs.split("\\")[-1],
                "documentType": document['documentType'],
                "offset": document['paragraphs'][0]['paragraphBody']['offset'],
                "text": "\n+++++++++++++\n".join(
                    str(x['paragraphBody']['text']) for x in document['paragraphs']),
                "length": sum(i['paragraphBody']['length'] for i in document['paragraphs']),
                "offsetHeader": document['paragraphs'][0]['paragraphHeader']['offset'],
                "textHeader": "\n+++++++++++++\n".join(
                    str(x['paragraphHeader']['text']) for x in document['paragraphs']),
                "lengthHeader": sum(i['paragraphHeader']['length'] for i in document['paragraphs'])
            })
    elif document['documentType'] == "SUPPLEMENTARY_AGREEMENT":
        flag = False
        obj = find_let(document)
        if obj != "":
            result.append(obj)
            flag = True

        if flag: continue
        #document['paragraphs'][0]['paragraphHeader']['text']
        result.append(
            {
                "path": docs,
                "name": docs.split("\\")[-1],
                "documentType": document['documentType'],
                "offset": document['paragraphs'][0]['paragraphBody']['offset'],
                "text": "".join(
                    str(x['paragraphBody']['text']) for x in document['paragraphs']),
                "length": sum(i['paragraphBody']['length'] for i in document['paragraphs']),
                "offsetHeader": document['paragraphs'][0]['paragraphHeader']['offset'],
                "textHeader": "\n+++++++++++++\n".join(
                    str(x['paragraphHeader']['text']) for x in document['paragraphs']),
                "lengthHeader": sum(i['paragraphHeader']['length'] for i in document['paragraphs'])
            })
    elif document['documentType'] == "AGREEMENT":
        flag = False
        for i, p in enumerate(document['paragraphs']):
            if any(f.lower() in p['paragraphHeader']['text'].lower() for f in
                   ['Предмет соглашения', 'Общие состоян', 'Общие положен']) and p['paragraphBody'][
                'length'] > 20:
                result.append(
                    {
                        "path": docs,
                        "name": docs.split("\\")[-1],
                        "documentType": document['documentType'],
                        "offset": p['paragraphBody']['offset'],
                        "text": p['paragraphBody']['text'],
                        "length": p['paragraphBody']['length'],
                        "offsetHeader": p['paragraphHeader']['offset'],
                        "textHeader": p['paragraphHeader']['text'],
                        "lengthHeader": p['paragraphHeader']['length']
                    })
                flag = True
                break
        if flag: continue

        obj = find_let(document)
        if obj != "":
            result.append(obj)
            flag = True

        if flag: continue

        # arr_of_paragraphs[
        #     paragraph_number if paragraph_number < len(arr_of_paragraphs) else 0][
        #     'paragraphBody']['text'] if
        # arr_of_paragraphs[paragraph_number if paragraph_number < len(
        #     arr_of_paragraphs) else 0]['paragraphBody']['length'] > 20 else "".join(
        #     str(x['paragraphBody']['text']) for x in document['paragraphs'])

        arr_of_paragraphs = document['paragraphs']
        result_of_fail.append(
            {
                "path": docs,
                "name": docs.split("\\")[-1],
                "documentType": document['documentType'],
                "offset": arr_of_paragraphs[0]['paragraphBody']['offset'],
                "text": "\n+++++++++++++\n".join(
                    str(x['paragraphBody']['text']) for x in document['paragraphs']),
                "length": sum(i['paragraphBody']['length'] for i in arr_of_paragraphs),
                "offsetHeader": arr_of_paragraphs[0]['paragraphHeader']['offset'],
                "textHeader": "\n+++++++++++++\n".join(
                    str(x['paragraphHeader']['text']) for x in document['paragraphs']),
                "lengthHeader": sum(i['paragraphHeader']['length'] for i in document['paragraphs'])
            })

#Смерть java процессу!
if platform.system() == 'Windows':
    subprocess.run("TASKKILL /F /PID {pid} /T".format(pid=java_subprocess.pid))
elif platform.system() == 'Linux':
    os.kill(java_subprocess.pid, signal.SIGTERM)
else:
    print('Не известная платформа, убейте в ручную процесс java')
# os.killpg(os.getpgid(java_subprocess.pid), signal.SIGTERM)

writer = pd.ExcelWriter("classifier.xlsx", engine='xlsxwriter')

df = pd.DataFrame(result)
df.to_excel(writer, 'good', engine='xlsxwriter')
sheets_good = writer.sheets['good']
sheets_good.autofilter(0, 0, df.shape[0], df.shape[1])

df = pd.DataFrame(result_of_fail)
df.to_excel(writer, 'SO SO', engine='xlsxwriter')
sheets_bad = writer.sheets['SO SO']
sheets_bad.autofilter(0, 0, df.shape[0], df.shape[1])

# df = pd.DataFrame(result_of_possible)
# df.to_excel(writer, 'SO SO SO', engine='xlsxwriter')
# sheets_bad = writer.sheets['SO SO SO']
# sheets_bad.autofilter(0, 0, df.shape[0], df.shape[1])

writer.save()
print("\nФайл создан")

Запуск document-parser на 8083 порту, если что0то пойдет не так, то руками УБЕЙТЕ java процесс
Проверка соединения #31
Готово
Запустился успешно
Общее количество документов = 1261
Проверка документа под номером 362
Ошибка в файле ДД по практикам\Практика правового сопровождения земельных отношений и сделок с недвижимым имуществом\Дополнительное соглашение N1 Г-РП.doc
Ответ от парсера {'timestamp': '2021-12-13T15:45:01.378+0000', 'status': 400, 'error': 'Bad Request', 'message': 'The document is really a OOXML file', 'path': '/document-parser', 'version': '1.6.4'}
Исключение = 'documents'
Проверка документа под номером 380
Ошибка в файле ДД по практикам\Практика правового сопровождения операционной деятельности БРД\Договоры\04-6.docx
Ответ от парсера {'timestamp': '2021-12-13T15:45:01.875+0000', 'status': 500, 'error': 'Internal Server Error', 'message': 'Cannot invoke "java.math.BigInteger.intValue()" because "abstractNumId" is null', 'path': '/document-parser', 'version': '1.6.4'}
Иск